# Balatro Poker Hand Notebook

Predict the probability of poker hands in a game of Balatro.

This notebook has miscellaneous sketches. The final script will be available in `balatro_stat.py`.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import json
import math

from balatro_save_reader import BalatroSaveReader, BalatroCard, BalatroPokerHand
from poker_hands import find_poker_hands, PokerHandName
from balatro_stat import next_hand_stats

Balatro uses a save format that I haven't seen before. Possibly something about Lua or the LOVE game engine? It looks like: `{["key"]=value,}`. This section explores the file format and parses it into a Python object for easier reading.

In [ ]:
save_file = 'Balatro\\1\\save.jkr'
save_path = os.path.join(os.getenv('APPDATA'), save_file)

save = BalatroSaveReader(save_path)

# Print the raw file
# print(str(save.balatro_save_file))

# Print the parsed dictionary
# print(json.dumps(save.data, indent=4))

print(f'Deck has {len(save.deck())} cards.')

print('Poker Hands:')
print(*save.poker_hands())

print('Current Hand:')
print(*save.hand())

print('Poker Hands:')
print(find_poker_hands(save.hand()))

# Poker Hand Testing

Yes, my unit tests live in this notebook. Thank you for asking.

In [ ]:
def make_cards(cards) -> list[BalatroCard]:
    value_dict = { 'A': 'Ace', 'K': 'King', 'Q': 'Queen', 'J': 'Jack' }
    suit_dict = { 'C': 'Clubs', 'D': 'Diamonds', 'H': 'Hearts', 'S': 'Spades' }
    def parse(card):
        value, suit = card[:-1], card[-1:]
        value = value_dict[value.upper()] if value.upper() in value_dict else value
        suit = suit_dict[suit.upper()] if suit.upper() in suit_dict else suit
        return BalatroCard(-1, {
            'base': {
                'value': value,
                'suit': suit
            },
            'ability': {
                'bonus': 0,
                'mult': 0,
            }
        })

    return [parse(card) for card in cards]

In [ ]:
def expect(cards: list[str], expected: list[str]):
    hand = make_cards(cards)
    poker_hands = find_poker_hands(hand)
    expected.append(PokerHandName.HIGH_CARD) # it's always there
    failed_tests = [hand_name for hand_name, is_found in poker_hands.items() if is_found != (hand_name in expected)]

    hand_str = ' '.join(map(str, hand))
    if failed_tests:
        print('FAIL', f'{hand_str:<42}', 'FAILED:', failed_tests)
    else:
        print('PASS', f'{hand_str:<42}', expected)

expect(['aS', 'qH', '10D', '8C', '6H', '4D', '3D', '2C'], [])
expect(['aS', 'kH', 'qD', 'jC', '10C', '4H', '3D', '2C'], [PokerHandName.STRAIGHT])
expect(['aC', 'kC', 'qC', 'jC', '10C'], [PokerHandName.STRAIGHT_FLUSH, PokerHandName.FLUSH, PokerHandName.STRAIGHT])
expect(['aS', 'kH', 'kS', 'qC', 'qS', 'jC', 'jS', '10S', '2C'], [PokerHandName.STRAIGHT_FLUSH, PokerHandName.FLUSH, PokerHandName.STRAIGHT, PokerHandName.TWO_PAIR, PokerHandName.PAIR])
expect(['aS', 'qH', '10D', '6C', '5C', '4H', '3D', '2C'], [PokerHandName.STRAIGHT])
expect(['aS', 'qH', '7C', '6D', '5D', '4D', '3D', '2C', '2D'], [PokerHandName.STRAIGHT_FLUSH, PokerHandName.FLUSH, PokerHandName.STRAIGHT, PokerHandName.PAIR])
expect(['9H', '9S', '4D', '4H', '7S', '7D', '7C'], [PokerHandName.FULL_HOUSE, PokerHandName.THREE_OF_A_KIND, PokerHandName.TWO_PAIR, PokerHandName.PAIR])
expect(['qH', '9H', '7S', '7H', '6H', '3H'], [PokerHandName.FLUSH, PokerHandName.PAIR])
expect(['4S', '4C', '4H', '4D'], [PokerHandName.FOUR_OF_A_KIND, PokerHandName.THREE_OF_A_KIND, PokerHandName.PAIR])
expect([], [])
expect(['5D', '3H'], [])

In [ ]:
hand = make_cards([ 'aS', 'qS', '10D', '6H', '6D', '5C', '3H', '3C'])
deck = make_cards([ 'kS', '6C', '6S', '9C', '9S', 'jS', 'jD', '4S', '2C'])

stats = next_hand_stats(hand, deck, max_discard=2)
print(json.dumps(stats, indent=3))

# Combinatorics

Combinatorics of brute forcing 'next hand' probabilities. How far can I brute force?

Seems like I'll be generally good with 3 discards - possibly up to 5 discards depending on how efficient things run.

In [ ]:
hand_size = 8
deck_size = 44

max_discard = 5

operation_count = 0
for discard_size in range(1, max_discard + 1):
    discard_combinations = math.comb(hand_size, discard_size)
    draw_combinations = math.comb(deck_size, discard_size)

    print(f'\t> Discarding {discard_size}: ({discard_combinations:_}) * ({draw_combinations:_})')
    operation_count += discard_combinations * draw_combinations

print()
print(f'Hand Size: {hand_size}; Deck Size: {deck_size}; Max Discards: {max_discard};')
print(f'Total Operations to Brute Force: {operation_count:_}')